In [1]:
import pandas as pd

In [2]:
gender = pd.read_csv("gender_submission.csv")
train = pd.read_csv("train.csv") #дата для тренировки
test = pd.read_csv("test.csv") #дата для проверки

In [9]:
train["Age"] = train["Age"].fillna(train["Age"].median())
train["Embarked"] = train["Embarked"].fillna("S")
test["Age"] = test["Age"].fillna(test["Age"].median())
test["Fare"] = test["Fare"].fillna(test["Fare"].median())

In [10]:
data_y = train[["Survived"]]
data_x = train[["Sex", "Pclass", "Age", "SibSp", "Parch", "Fare", "Embarked"]]

test = test[["Sex","Pclass", "Age", "SibSp", "Parch", "Fare", "Embarked"]]

In [11]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False).set_output(transform='pandas')
ohetranseform = ohe.fit_transform(data_x[["Sex", "Embarked"]]) #we have to put in a dataframe with [[]]
data_x = pd.concat([data_x, ohetranseform], axis=1).drop(columns = ["Sex", "Embarked"])

In [12]:
ohetranseform = ohe.fit_transform(test[["Sex", "Embarked"]])
test = pd.concat([test, ohetranseform], axis=1).drop(columns = ["Sex", "Embarked"])

In [13]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(data_x, data_y, train_size=0.8, random_state=5)

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [15]:
params = {
    'criterion':  ['log_loss', 'entropy'],
    'max_depth':  [None, 2, 4, 6, 8, 10],
    'max_features': [None, 'sqrt', 'log2', 0.2, 0.4, 0.6, 0.8],
    'splitter': ['best', 'random']
}

clf = GridSearchCV(
    estimator=DecisionTreeClassifier(),
    param_grid=params,
    cv=5,
    n_jobs=5,
    verbose=1,
)


In [16]:
clf = clf.fit(train_x, train_y)

Fitting 5 folds for each of 168 candidates, totalling 840 fits


In [17]:
pred = clf.predict(test_x)
accuracy_score(test_y, pred)

0.8212290502793296

In [18]:
print (clf.best_params_)
clf.best_score_

{'criterion': 'log_loss', 'max_depth': 10, 'max_features': 0.4, 'splitter': 'random'}


0.8173544765094063

In [22]:
clf_best = DecisionTreeClassifier(criterion="log_loss", max_depth=10, max_features=0.4, splitter="random")

In [23]:
clf_best.fit(train_x, train_y)
predict = clf_best.predict(test)

index = []
for number in range(892, 1310):
    index.append(number)

In [24]:
df = pd.DataFrame(data = predict, index = index, columns=["Survived"])
df.index.name = "Passenger_id"
results = df.to_csv("Results.csv")